In [1]:
from transformers import AutoTokenizer, LayoutLMForQuestionAnswering
from datasets import load_dataset
import torch

tokenizer = AutoTokenizer.from_pretrained(
    "impira/layoutlm-document-qa", add_prefix_space=True
)
model = LayoutLMForQuestionAnswering.from_pretrained(
    "impira/layoutlm-document-qa", revision="1e3ebac"
)

dataset = load_dataset("nielsr/funsd", split="train")

/Users/jieyinuo/opt/anaconda3/envs/innocent_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 993/993 [00:00<00:00, 238kB/s]
pytorch_model.bin: 100%|██████████| 511M/511M [00:35<00:00, 14.6MB/s] 
/Users/jieyinuo/opt/anaconda3/envs/innocent_project/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for nielsr/funsd contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nielsr/funsd
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 149 examples [00:01, 119.41 examples/

M. Hamann P. Harper, P. Martinez


In [1]:
from transformers import pipeline

recog = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa",
)

/Users/jieyinuo/opt/anaconda3/envs/innocent_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
question = "What is the name of applicant?"
recog("/Users/jieyinuo/Desktop/hackathon/1.png", question)

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [1]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
ocr = PaddleOCR(
    use_angle_cls=True, lang="en", page_num=2
)  # need to run only once to download and load model into memory

: 

In [ ]:
img_path = "/Users/jieyinuo/Desktop/hackathon/1.png"
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)

In [ ]:
# 显示结果
import fitz
from PIL import Image
import cv2
import numpy as np

imgs = []
with fitz.open(img_path) as pdf:
    for pg in range(0, pdf.pageCount):
        page = pdf[pg]
        mat = fitz.Matrix(2, 2)
        pm = page.getPixmap(matrix=mat, alpha=False)
        # if width or height > 2000 pixels, don't enlarge the image
        if pm.width > 2000 or pm.height > 2000:
            pm = page.getPixmap(matrix=fitz.Matrix(1, 1), alpha=False)

        img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        imgs.append(img)
for idx in range(len(result)):
    res = result[idx]
    image = imgs[idx]
    boxes = [line[0] for line in res]
    txts = [line[1][0] for line in res]
    scores = [line[1][1] for line in res]
    im_show = draw_ocr(image, boxes, txts, scores, font_path="doc/fonts/simfang.ttf")
    im_show = Image.fromarray(im_show)
    im_show.save("result_page_{}.jpg".format(idx))